In [1]:
from sklearn.datasets import load_digits
from sklearn.decomposition import FastICA
from sklearn.decomposition import PCA
import pickle
import copy

import pandas as pd
import seaborn as sns
import numpy
import torch
import scipy
import scipy.stats
import numpy as np

%matplotlib widget
import matplotlib.pyplot as plt

In [2]:
import os
dim=3
base_dir='/Users/phumon/Documents/Research/nlu-test-sets/params_snli'
file_name=f'alpha-lognormal-identity-dim3_theta-normal-identity_nosubsample_1.00_0.50'
exp_dir=os.path.join(base_dir, file_name)

print(exp_dir)


/Users/phumon/Documents/Research/nlu-test-sets/params_snli/alpha-lognormal-identity-dim3_theta-normal-identity_nosubsample_1.00_0.50


In [3]:
param_plot_stats = {}
keys = ['a', 'b', 'g', 't']
for key in keys:
    with open(os.path.join(exp_dir, 'plot_stats_pickles', f'{key}.p'), 'rb') as f:
        param_plot_stats[key] = pickle.load(f)

In [4]:

#param_plot_stats['a']['mean'] = param_plot_stats['a']['mean'].apply(lambda x: numpy.log(x))
alpha_mean, alpha_var = param_plot_stats['a']["mean"].to_numpy().mean(), param_plot_stats['a']["var"].to_numpy().mean()
print(alpha_mean, alpha_var)

[0.89607114 0.89446325 0.89290299] [0.16615165 0.16537487 0.16492233]


In [5]:
dimensions=3
for dim in range(dimensions):
    param_plot_stats['a']['mean_'+str(dim)]= param_plot_stats['a']['mean'].apply(lambda x: x[dim])
    param_plot_stats['b']['mean_'+str(dim)]= param_plot_stats['b']['mean'].apply(lambda x: x[dim])

In [6]:
plt.figure()
for dim in range(dimensions):
    plt.plot(param_plot_stats['a']['mean_'+str(dim)], param_plot_stats['b']['mean_'+str(dim)], marker="o", linestyle="", label=dim)

plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.figure()
plt.plot(param_plot_stats['a']['mean_0'], param_plot_stats['a']['mean_1'], marker="o", linestyle="", label=1)
plt.plot(param_plot_stats['a']['mean_0'], param_plot_stats['a']['mean_2'], marker="o", linestyle="", label=2)

plt.legend()
plt.show()

In [ ]:
param_plot_stats['a']['mean']

In [ ]:
plt.figure()
plt.plot(param_plot_stats['a']['mean_0'], param_plot_stats['a']['mean_2'], marker="o", linestyle="")
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(param_plot_stats['b']['mean_0'], param_plot_stats['b']['mean_1'], marker="o", linestyle="", label=1)
plt.plot(param_plot_stats['b']['mean_0'], param_plot_stats['b']['mean_2'], marker="o", linestyle="", label=2)
plt.legend()
plt.show()

https://stackoverflow.com/questions/35407669/independent-component-analysis-ica-in-python

In [ ]:
alpha_arr = np.vstack(param_plot_stats['b']['mean'])
pca = PCA(n_components=2, random_state=0)
new_alpha_ = pca.fit_transform(alpha_arr)  # Reconstruct signals
#A_alpha_ = ica.mixing_  # Get estimated mixing matrix


In [ ]:
print(new_alpha_.shape)

In [ ]:
from multi_virt_v2 import *

In [ ]:
repo = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))

In [ ]:
#datasets="boolq,cb,commonsenseqa,copa,cosmosqa,hellaswag,rte,snli,wic,qamr,arct,mcscript,mctaco,mutual,mutual-plus,quoref,socialiqa,squad-v2,wsc,mnli,mrqa-nq,newsqa,abductive-nli,arc-easy,arc-challenge,piqa,quail,winogrande,anli"
datasets="snli"
data_names, responses, n_items = get_files(
    os.path.join(repo, 'data'),
    "csv",
    set(datasets.split(','))
)

In [ ]:
task_metadata = pd.read_csv('../irt_scripts/task_metadata.csv')
task_metadata.set_index("jiant_name", inplace=True)
task_list = [x for x in task_metadata.index if x in data_names]

In [ ]:
total = 0
task_name = []
task_format = []

for tname, size in zip(data_names, n_items):
    name = task_metadata.loc[tname]['taskname']
    total += size
    task_name += [name for _ in range(size)]
    task_format += [task_metadata.loc[tname]['format'] for _ in range(size)]
    
task_name = pd.DataFrame(task_name, columns=['task_name'])
task_format = pd.DataFrame(task_format, columns=['format'])
task_name_format = pd.concat([task_name, task_format], axis=1)

In [ ]:
task_name_format

In [ ]:

param = pd.concat([pd.DataFrame(new_alpha_.tolist()), task_name_format], axis=1)

In [ ]:
param

In [ ]:


plt.figure()
plt.plot(param[0], param[1], "kx")
plt.show()




In [ ]:
groups = param.groupby("task_name")
groups

In [ ]:
plt.figure()

groups = param.groupby("format")
for name, group in groups:
    print("name: ",name)
    plt.plot(group[0], group[1], marker="o", linestyle="")
plt.legend()
plt.show()

In [ ]:
param[0]

# Theta

In [ ]:
beta_arr = np.vstack(param_plot_stats['b']['mean'])
pca = PCA(n_components=2, random_state=0)
new_beta_ = pca.fit_transform(beta_arr)  # Reconstruct signals
#A_alpha_ = ica.mixing_  # Get estimated mixing matrix


In [ ]:
params_beta = pd.concat([pd.DataFrame(new_beta_.tolist()), task_name_format], axis=1)

In [ ]:
groups = params_beta.groupby("task_name")


In [ ]:
plt.figure()
for name, group in groups:
    print("name: ",name)
    plt.plot(group[0], group[1], marker="o", linestyle="", label=name)
plt.legend()
plt.show()

In [ ]:
# END scatter plot

In [ ]:
beta_arr = np.vstack(param_plot_stats['b']['mean'])
ica = FastICA(n_components=3, random_state=0)
S_beta_ = ica.fit_transform(beta_arr)  # Reconstruct signals
A_beta_ = ica.mixing_  # Get estimated mixing matrix


In [ ]:
models = [beta_arr, S_beta_]
names = ['Alpha (mixed signal)',
         'ICA recovered Alpha signals']
colors = ['red', 'steelblue', 'orange']

plt.figure()
for ii, (model, name) in enumerate(zip(models, names), 1):
    plt.subplot(2, 1, ii)
    plt.title(name)
    for sig, color in zip(model.T, colors):
        plt.plot(sig, color=color)

plt.subplots_adjust(0.09, 0.04, 0.94, 0.94, 0.26, 0.46)
plt.show()

In [ ]:
params_beta = pd.concat([pd.DataFrame(S_beta_.tolist()), task_name_format], axis=1)

In [ ]:
params_beta

In [ ]:
groups = params_beta.groupby("task_name")
groups

In [ ]:
plt.figure()
for name, group in groups:
    print("name: ",name)
    plt.plot(group[0], group[1], marker="o", linestyle="", label=1)
    plt.plot(group[0], group[2], marker="o", linestyle="", label=2)
plt.legend()
plt.show()